In [145]:
"""
скрипт для обработки данных мониторинга трудоустройства
"""


'\nскрипт для обработки данных мониторинга трудоустройства\n'

In [146]:
import pandas as pd
import numpy as np
import openpyxl
import warnings
import copy

warnings.simplefilter(action='ignore', category=UserWarning)

In [147]:
path_to_file ='data/Брит - 2022.xlsx'
path_to_end_folder = 'data/temp'
name_file = 'БРИТ'

In [148]:
df = pd.read_excel(path_to_file,skiprows=7,dtype=str)

In [149]:
high_level_dct = {'БРИТ':{}}

In [150]:
df.columns = list(map(str,df.columns))

In [151]:
df.head()

,01,02,03,04,05,06,07,08,09,10,...,25,26,27,28,29,30,31,32,33,34
0,ДФО,Республика Бурятия,43.01.06,Проводник на железнодорожном транспорте,01,Всего (общая численность выпускников),21,8,7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,проверка пройдена
1,ДФО,Республика Бурятия,NaN,NaN,02,из общей численности выпускников (из строки 01...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,проверка пройдена
2,ДФО,Республика Бурятия,NaN,NaN,03,из числа лиц с ОВЗ (из строки 02): инвалиды и ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,проверка пройдена
3,ДФО,Республика Бурятия,NaN,NaN,04,Инвалиды и дети-инвалиды (кроме учтенных в стр...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,проверка пройдена
4,ДФО,Республика Бурятия,NaN,NaN,05,Имеют договор о целевом обучении,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,проверка пройдена


In [152]:
# Заполняем пока пропуски в 15 ячейке для каждой специальности
df['06'] = df['06'].fillna('15 ячейка')

In [153]:
df['03'].unique()

array(['43.01.06', nan, '43.02.06', '23.01.09', ' ', '23.02.06',
       '23.01.03', '15.01.05', '15.01.35', '15.01.33', '15.01.26',
       '23.01.10', '38.02.02', '08.02.09', '23.02.03', '08.02.08',
       '43.01.09', '08.01.18', '13.01.10'], dtype=object)

In [154]:
# очищаем от нан и возможнных пустых пробелов
code_spec =[spec for spec in df['03'].unique() if spec is not np.nan]
code_spec =[spec for spec in code_spec if spec != ' ']

In [155]:
code_spec

['43.01.06',
 '43.02.06',
 '23.01.09',
 '23.02.06',
 '23.01.03',
 '15.01.05',
 '15.01.35',
 '15.01.33',
 '15.01.26',
 '23.01.10',
 '38.02.02',
 '08.02.09',
 '23.02.03',
 '08.02.08',
 '43.01.09',
 '08.01.18',
 '13.01.10']

In [156]:
# Создаем список для строк
row_cat = [f'Строка {i}' for i in range(1,16)]
# Создаем список для колонок
column_cat = [f'Колонка {i}' for i in range(7,34)]

In [157]:
#iddle_dct = dict.fromkeys(row_cat,{key:0 for key in column_cat}.copy())

In [158]:
# Создаем словарь нижнего уровня содержащий в себе все данные для каждой специальности
spec_dict = {}
for row in row_cat:
    spec_dict[row] = {key:0 for key in column_cat}

In [159]:
id(spec_dict['Строка 1'])

2243549057344

In [160]:
id(spec_dict['Строка 2'])

2243549796928

In [171]:
# Создаем словарь среднего уровня содержащй данные по всем специальностям
poo_dct = {key:copy.deepcopy(spec_dict) for key in code_spec}

In [172]:
id(poo_dct['15.01.33']['Строка 1'])

2243553361728

In [173]:
id(poo_dct['13.01.10']['Строка 1'])

2243500899904